## Santa Barbara Weather Forecast Model Evaluation
### UCSB Climate Variation and Changes Research Group
#### Advisor: Professor Charles Jones
#### Author: Pippa Lin

In [2]:
import numpy as np
import pandas as pd
import altair as alt
import math
import os

### Introduction:
This research project aims to interpret present and future climates in Santa Barbara from the Numerical Weather Forecast (NWF) model. The data is collected from https://clivac.eri.ucsb.edu/clivac/wrfreal/index.html. In this project, I performed statistical analysis to compare NWF model data and weather station observation, generating root mean square error, mean bias, and correlation by hour to assess model performance.

### 1.Projection Function
This function combines wind speed and wind direction and projects it onto two perpendicular values: zonal and meridional.

In [20]:
def projection(direction, speed):
    zonal = 0  # Default value for zonal
    meridional = 0
    
    if (speed == 9999. or direction == 9999.):
        zonal = 9999
        meridional = 9999
     
    elif direction == 360.0:
        direction == 0.0
        
    elif (direction >= 0 and direction < 90):
        zonal = -speed * np.sin(direction)
        meridional = -speed * np.cos(direction)
    
    elif (direction >= 90 and direction < 180):
        beta = 180 - direction
        zonal = -speed * np.sin(beta)
        meridional = speed * np.cos(beta)
        
    elif (direction >= 180 and direction < 270):
        beta  = 270.0 - direction
        zonal =  speed * np.cos(beta)
        meridional =  speed * np.sin(beta)
    
    elif (direction >= 270 and direction < 360):
        beta = 360 - direction
        zonal = speed * np.sin(beta)
        meridional = -speed * np.cos(beta)
    
    else:
        zonal = np.nan
        meridional = np.nan

    return zonal, meridional



def projectionList(Listdirection, Listspeed):
    Listzonal = []
    Listmeridional = []
    for d,s in zip(Listdirection, Listspeed):
        Listzonal.append(projection(d,s)[0])
        Listmeridional.append(projection(d,s)[1])
    return Listzonal, Listmeridional

### 2.Read the data:
Read and tidy the data

In [4]:
folder_path = '/Users/pippalin/Desktop/Climate Research/xskill-mtic1/' 
file_names = os.listdir(folder_path)

# Initialize an empty list to store the data frames
dfs = []

# Loop through the list of file names and read each file into a Pandas DataFrame
for file_name in file_names:
    # Check if the file is a CSV file
    if file_name.endswith('.txt'):
        # Read the file into a Pandas DataFrame
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, skiprows = 7,nrows = 73) # since files with > 73 rows has nan values after row 73
        # Change the column name of the data frame
        df.set_axis(['yyyy','mm', 'dd', 'hh', 'min','ss','modtemp','modrh','modwsp','modwd','yyyy1','mm1', 'dd1', 'hh1', 'mm1','ss1','obstemp','obsrh','obswsp','obswd'], axis=1, inplace=True)
        
        # Frist, move the obs-part up by one, covering the first row, and drop the last row:
        df.loc[0:len(df)-2, 'yyyy1':'obswd'] = df.loc[1:len(df)-1, 'yyyy1':'obswd'].values
        df = df.drop(df.index[-1])

        # Add a column hour
        df["hour"] = range(len(df))
        
        dfs.append(df)
        
# Concatenate the data frames into a single data frame
merged_df = pd.concat(dfs).sort_values(by=['yyyy','mm']).reset_index(drop=True)

### 3.Create the Zonal and Meridional columns:

In [31]:
merged_df["modzonal"],merged_df["modmeridional"] = projectionList(merged_df["modwd"], merged_df["modwsp"])
merged_df["obszonal"],merged_df["obsmeridional"] = projectionList(merged_df["obswd"], merged_df["obswsp"])

### 4.Calculate RMSE, MB, Correlation by hour

#### RMSE:
1. Calcualte $(mod-obs)^2$
2. Group by hour and sum up $(mod-obs)^2$
3. Divide the sum by n and take square root

In [33]:
# 1. Calculate the square difference
merged_df["zonal_dif"] = (merged_df["obszonal"] - merged_df["modzonal"])**2
merged_df["meridional_dif"] = (merged_df["obsmeridional"] - merged_df["modmeridional"])**2

In [37]:
# 2. Group by hour and sum
sum_zonal = merged_df.groupby("hour")["zonal_dif"].sum()
sum_meridional = merged_df.groupby("hour")["meridional_dif"].sum()

In [38]:
# 3. Divide the sums by n and take square root, calculate rmse and make it a dataframe
count = merged_df.groupby("hour").count()

# First, write a function to calculate rmse and return rmse
def rmse(data,count):
    rmse = []
    for i in range(len(data)):
        rmse.append(np.sqrt(data[i]/count[i]))
    rmse = pd.DataFrame(rmse,columns = ['rmse'])
    rmse['hour'] = range(0,72)
    return rmse

# Now we calculate rmse for each variable
rmse_zonal = rmse(sum_zonal,count["obszonal"])
rmse_meridional = rmse(sum_meridional,count["obsmeridional"])

#### Write a funtcion of plotting

In [39]:
def plotrmse(dataframe,plotname):
    # A straight line
    rule = alt.Chart(pd.DataFrame({'Component': [24,48,72]})).mark_rule(color='#D2386C').encode(x='Component')
    
    # Plotting
    plt = alt.Chart(dataframe).mark_line().encode(
    x = 'hour:Q',
    y = 'rmse',
    color=alt.value("#FFAA00")
).properties(
    width = 800,
    height = 300,
    title={
      "text": [plotname],
      "color": "green"
    })
    return plt + rule

In [40]:
plotrmse(rmse_zonal,"RMSE plot of Zonal")

alt.LayerChart(...)

In [41]:
plotrmse(rmse_meridional,"RMSE plot of Meridional")

alt.LayerChart(...)

<br>

#### MB:
1. Calcualte $(mod-obs)$
2. Group by hour and sum up $(mod-obs)$
3. Divide the sum by n

In [42]:
# 1. Calculate the difference
merged_df["zonal_dif"] = merged_df["obszonal"] - merged_df["modzonal"]
merged_df["meridional_dif"] = merged_df["obsmeridional"] - merged_df["modmeridional"]

In [43]:
# 2. Group by hour and sum
sum_zonal = merged_df.groupby("hour")["zonal_dif"].sum()
sum_meridional = merged_df.groupby("hour")["meridional_dif"].sum()

In [44]:
# 3. Divide the sums by n and take square root, calculate rmse and make it a dataframe
count = merged_df.groupby("hour").count()

# First, write a function to calculate rmse and return rmse
def mb(data,count):
    mb = []
    for i in range(len(data)):
        mb.append(data[i]/count[i])
    mb = pd.DataFrame(mb,columns = ['mb'])
    mb['hour'] = range(0,72)
    return mb

# Now we calculate rmse for each variable
mb_zonal = mb(sum_zonal,count["obszonal"])
mb_meridional = mb(sum_meridional,count["obsmeridional"])

#### Write a funtcion of plotting

In [45]:
def plotmb(dataframe,plotname):
    # A straight line
    rule = alt.Chart(pd.DataFrame({'Component': [24,48,72]})).mark_rule(color='#D2386C').encode(x='Component')
    
    # Plotting
    plt = alt.Chart(dataframe).mark_line().encode(
    x = 'hour:Q',
    y = 'mb',
    color=alt.value("#FFAA00")
).properties(
    width = 800,
    height = 300,
    title={
      "text": [plotname],
      "color": "green"
    })
    return plt + rule

In [47]:
plotmb(mb_zonal,"MB plot of Zonal")

alt.LayerChart(...)

In [48]:
plotmb(mb_meridional,"MB plot of Meridional")

alt.LayerChart(...)

<br>

#### Correlation:
1. Create a matrix with:
$$\begin{bmatrix} [mod0h] & [mod1h] & ... & [mod84h] \\ [obs0h] & [obs1h] & ... & [obs84h] \end{bmatrix}$$
where [mod0h] contains a list of all values from 0h
* Note: If the list contains None type value, then the correlation will be None. Therefore, we need to remove the null value in a list and also remove the corresponding obs/mod value. We can write a function to do so:
2. Calculate correlation function between mod_nh and obs_nh

In [50]:
# 1. Define a function which create lists with all values of that hour, also removing null values
def create_list(var):
    return merged_df.groupby("hour")[var].apply(lambda x: x.tolist())

# Calculate the list
modzonal_list = create_list("modzonal")
obszonal_list = create_list("obszonal")
modmeridional_list = create_list("modmeridional")
obsmeridional_list = create_list("obsmeridional")

In [52]:
# 2. Since there are NaN for each list, we need to remove nan value and the corresponding index, 
# we first record the index of the missing value and remove i
def index(modlist,obslist):
    # Get the index matrix
    index = [[] for _ in range(len(modlist))]
    for i in range(len(modlist)):
        for j in range(len(modlist[i])):
            if pd.isna(modlist[i][j]) or pd.isna(obslist[i][j]):
                index[i].append(j)
    return index

In [53]:
# 3. Remove the corresponding value from the index
def removeNan(index,modlist,obslist):
    update_mod = [[] for _ in range(len(modlist))]
    update_obs = [[] for _ in range(len(obslist))]
    
    for i in range(len(modlist)):
        for j in range(len(modlist[i])):
            if j not in index[i]:
                update_mod[i].append(modlist[i][j])
                update_obs[i].append(obslist[i][j])
    return update_mod,update_obs

In [54]:
up_modzonal_list,up_obszonal_list = removeNan(index(modzonal_list,obszonal_list),modzonal_list,obszonal_list)
up_modmeridional_list,up_obsmeridional_list = removeNan(index(modmeridional_list,obsmeridional_list),modmeridional_list,obsmeridional_list)

In [55]:
# 4. Create a function to calculate correlation and create a list
def correlation(modlist,obslist):
    correlation = []
    for i in range(len(modlist)):
        correlation.append(np.corrcoef(modlist[i],obslist[i])[0][1])
        
    # Turn into dataframe
    correlation = pd.DataFrame(correlation,columns = ['corr'])
    correlation['hour'] = range(0,72)
    return correlation

In [56]:
corr_zonal = correlation(up_modzonal_list,up_obszonal_list)
corr_meridional = correlation(up_modmeridional_list,up_obsmeridional_list)

#### Write a function for plotting

In [68]:
def plotcorr(dataframe,plotname):
    # A straight line
    rule = alt.Chart(pd.DataFrame({'Component': [24,48,72]})).mark_rule(color='#D2386C').encode(x='Component')
    
    # Plotting
    plt = alt.Chart(dataframe).mark_line().encode(
    x = 'hour:Q',
    y = 'corr',
    color=alt.value("#FFAA00")
).properties(
    width = 800,
    height = 300,
    title={
      "text": [plotname],
      "color": "green"
    })
    return plt + rule

In [69]:
plotcorr(corr_zonal,"Correlation plot of Zonal")

alt.LayerChart(...)

In [60]:
plotcorr(corr_meridional,"Correlation plot of Meridional")

alt.LayerChart(...)